<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/DataScience/0614GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Search and Cross Validation

모델링의 가장 최적의 하이퍼파라미터 값을 찾기 위해, 일일이 모두 다 해보는 방법인 Grid Search에 대해서 수행 방법을 알아보자.

또한 Stratified K-fold 및 위를 한번에 다하는 cross_val_score를 알아보자.

# K-fold vs Stratified K-fold

그냥 K-fold와 다르게 층화, Stratified K-fold는 train에서의 label 분포와 test에서의 label 분포를 유사하게 한다.

In [3]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5)
all_acc = []
fold_idx = 0

iris = load_iris()
features = iris.data
labels = iris.target

for train_idx, test_idx in skf.split(features, labels):
  train_x, train_y = features[train_idx], labels[train_idx]
  test_x, test_y = features[test_idx], labels[test_idx]

  model = DecisionTreeClassifier()
  model.fit(train_x, train_y)
  pred_y = model.predict(test_x)
  acc = accuracy_score(test_y, pred_y)

  fold_idx += 1
  all_acc.append(acc)

print(np.mean(all_acc))

0.9600000000000002


그런데, 이를 한번에 해 줄 코드가 존재한다. cross_val_score을 통해 한번에 된다.

In [4]:
# Stratified K-fold 기반이다.
from sklearn.model_selection import cross_val_score, cross_validate

iris_data = load_iris()
features = iris_data.data
labels = iris.target

model = DecisionTreeClassifier()

scores = cross_val_score(model, features, labels,
                        scoring='accuracy', cv=5)
print(scores)

[0.96666667 0.96666667 0.9        0.96666667 1.        ]


## GridSearch
그리드서치란, 모델링시 필요한 하이퍼파리미터를 설정할 때, 가장 최적의 파라미터 값을 찾아주는 방법이다.  
파라미터 후보값을 사전에 정의해주고 이를 일일이 for문 돌리지 말고, 이걸 대신 해준다. 거기에 교차검증까지 해주니, 그냥 이걸 하면 된다.



In [7]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

iris_data = load_iris()
train_x, test_x, train_y, test_y = train_test_split(iris_data.data, iris_data.target,
                                                    test_size = 0.2)
model = DecisionTreeClassifier()
params = {'max_depth':[3, 4, 5, 6], 'min_samples_split':[2, 3, 4]}

grid_dtree = GridSearchCV(model, param_grid = params,
                          cv = 5, refit = True, return_train_score = True)
grid_dtree.fit(train_x, train_y)

# 결과표 출력
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

# best_params_ 할당
print(f"최적의 파라미터 : {grid_dtree.best_params_}")
print(f"최적의 파라미터로 모델의 정확도 : {grid_dtree.best_score_}")

# 최적의 best_params
estimator = grid_dtree.best_estimator_
pred_y = estimator.predict(test_x)
print(accuracy_score(test_y, pred_y))


최적의 파라미터 : {'max_depth': 3, 'min_samples_split': 2}
최적의 파라미터로 모델의 정확도 : 0.975
0.9666666666666667
